In [ ]:
!pip install torch torchaudio transformers phonemizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 5.6 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10

In [ ]:
!apt-get install espeak-ng -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 21 not upgraded.
Need to get 3,957 kB of archives.
After this operation, 10.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpcaudio0 amd64 1.0-1 [6,536 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 espeak-ng-data amd64 1.49.2+dfsg-1 [3,469 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libespeak-ng1 amd64 1.49.2+dfsg-1 [187 kB]
Ge

In [182]:
import torch
from transformers import AutoModel, AutoTokenizer

repo = "Bingsu/torchaudio_tacotron2_kss"
model = AutoModel.from_pretrained(
    repo,
    trust_remote_code=True,
    revision="589d6557e8b4bb347f49de74270541063ba9c2bc"
    )
tokenizer = AutoTokenizer.from_pretrained(repo)
model.eval()

Tacotron2Model(
  (tacotron2): Tacotron2(
    (embedding): Embedding(392, 512)
    (encoder): _Encoder(
      (convolutions): ModuleList(
        (0): Sequential(
          (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
          (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
          (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): Sequential(
          (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
          (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
    )
    (decoder): _Decoder(
      (prenet): _Prenet(
        (layers): ModuleList(
          (0): Linear(in_features=80, out_features=256, bias=False)
         

In [334]:
vocoder = torch.hub.load("seungwonpark/melgan:aca59909f6dd028ec808f987b154535a7ca3400c", "melgan", trust_repo=True, pretrained=False)
url = "https://huggingface.co/Bingsu/torchaudio_tacotron2_kss/resolve/main/melgan.pt"
state_dict = torch.hub.load_state_dict_from_url(url)
vocoder.load_state_dict(state_dict)
vocoder.eval()

Using cache found in /root/.cache/torch/hub/seungwonpark_melgan_aca59909f6dd028ec808f987b154535a7ca3400c


In [345]:
text = "모든 창에서 촛불이 반짝반짝 빛나고 있었다."
inp = tokenizer(text, return_tensors="pt", return_length=True, return_attention_mask=False)

In [346]:
inp

{'input_ids': tensor([[ 13,  19,  12, 120,   4,   3,  66,  39,  20,  42,  16,   5,  33,   3,
          66,  39,  19,   6,  18,  34,  15,  10,   3,  18,  20,   4,  84,  20,
         101,  18,  20,   4,  84,  20, 101,   3,  18,  10,   4,   4,  20, 101,
          19,   3,  10,   5,  33,   6,   6,  20,   3]]), 'length': tensor([51])}

In [347]:
s = tokenizer.decode(inp['input_ids'][0])
print(s)

m o d ɯ n <unk> tʃ h ɐ ŋ e s ʌ <unk> tʃ h o t p u ɾ i <unk> p ɐ n tɕ ɐ q p ɐ n tɕ ɐ q <unk> p i n ɐ q o <unk> i s ʌ t ɐ <unk>


In [348]:
with torch.inference_mode():
    out = model(**inp)
    print(out[0].shape)
    audio = vocoder(out[0])
    print(audio.shape)

torch.Size([1, 80, 497])
torch.Size([1, 1, 127232])


In [349]:
import IPython.display as ipd

ipd.Audio(audio[0][0].numpy(), rate=22050)